In [1]:
import numpy as np
from os import listdir
from os.path import isfile, join
import os

In [2]:
# On local computer
dir_path = "D:/Users/leoca/Documents/Cours/Projet_3A/Dataset_couleur"
os.chdir(dir_path)

In [3]:
with open('array_entree_224.npy', 'rb') as f:
    X = np.load(f)

with open('array_sortie_224.npy', 'rb') as f:
    y = np.load(f)
    
train_proportion = int(0.8*len(X))

X_train = X[:train_proportion]
X_test = X[train_proportion:]
y_train = y[:train_proportion]
y_test = y[train_proportion:]

In [4]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pickle

# Définir une fonction qui crée votre modèle en tant qu'objet KerasClassifier
def create_model(learning_rate=0.001):
    classifier = Sequential()

    classifier.add(Conv2D(64, (3, 3), input_shape=(224, 224, 3), activation='relu'))
    classifier.add(Conv2D(64, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2),strides=2))

    classifier.add(Conv2D(128, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    classifier.add(Conv2D(256, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2),strides=2))

    classifier.add(Conv2D(512, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2),strides=2))
    classifier.add(Conv2D(512, (3, 3), activation='relu'))
    classifier.add(MaxPooling2D(pool_size=(2, 2),strides=2))

    classifier.add(Flatten())
    classifier.add(Dense(256, activation='relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(128, activation='relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(1, activation='sigmoid'))

    classifier.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

# Créer un objet KerasClassifier pour l'utilisation avec GridSearchCV
model = KerasClassifier(build_fn=create_model)

# Définir les hyperparamètres à explorer
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
learning_rate = [0.001, 0.0001]

# Créer le dictionnaire des hyperparamètres
param_grid = dict(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate)

# Créer l'objet GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3)

# Exécuter GridSearchCV sur les données d'entraînement
grid_result = grid.fit(X_train, y_train)

# Enregistrer l'objet grid_result dans un fichier
with open('grid_result.pkl', 'wb') as f:
    pickle.dump(grid_result, f)


C:\Users\leoca\AppData\Local\Temp\ipykernel_19020\601728949.py:30: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


MemoryError: Unable to allocate 877. MiB for an array with shape (1527, 224, 224, 3) and data type float32